In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
synthetic_df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/synthetic_data_adult_census.csv')


In [2]:
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race','sex', 'native_country']
for column in categorical_columns:
    tempdf = pd.get_dummies(synthetic_df[column], prefix=column)
    synthetic_df = pd.merge(
        left=synthetic_df,
        right=tempdf,
        left_index=True,
        right_index=True,
    )
    synthetic_df = synthetic_df.drop(columns=column)

In [3]:
x = synthetic_df.iloc[:, :-1].to_numpy()
y = synthetic_df['<50k'].to_numpy()

In [4]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.base import clone, BaseEstimator

try:
    import tensorflow as tf
except (ModuleNotFoundError, ImportError):
    import warnings

    warnings.warn("Tensorflow is not installed")

class ShadowModels:
    """
    Creates a swarm of shadow models and trains them with a split
    of the synthetic data.
    Parameters
    ----------
    X: ndarray or DataFrame
    y: ndarray or str
        if X it's a DataFrame then y must be the target column name,
        otherwise 
    n_models: int
        number of shadow models to build. Higher number returns
        better results but is limited by the number of records 
        in the input data.
    target_classes: int
        number of classes of the target model or lenght of the
        prediction array of the target model.
    learner: learner? #fix type
        learner to use as shadow model. It must be as similar as 
        possible to the target model. It must have `predict_proba` 
        method. Now only sklearn learners are implemented.
    Returns
    -------
    ShadowModels object
    """

    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        n_models: int,
        target_classes: int,
        learner,
        **fit_kwargs,
    ) -> None:

        self.n_models = n_models
        self.X = X
        if self.X.ndim > 1:
            # flatten images or matrices inside 1rst axis
            self.X = self.X.reshape(self.X.shape[0], -1)

        self.y = y
        self.target_classes = target_classes
        self._splits = self._split_data(self.X, self.y, self.n_models, self.target_classes)
        self.learner = learner
        self.models = self._make_model_list(self.learner, self.n_models)

        # train models
        self.results = self.train_predict_shadows(**fit_kwargs)

    @staticmethod
    def _split_data(
        X: np.ndarray, y: np.ndarray, n_splits: int, n_classes: int
    ) -> List[np.ndarray]:
        """
        Split manually into n datasets maintaining class proportions
        """
        # data = np.hstack((data[0], data[1].reshape(-1, 1)))
        # X = data
        # y = data[:, -1]
        classes = range(n_classes)
        class_partitions = []
        # Split by class
        for clss in classes:

            X_clss = X[y == clss]
            y_clss = y[y == clss]
            batch_size = len(X_clss) // n_splits
            splits = []
            for i in range(n_splits):
                split_X = X_clss[i * batch_size : (i + 1) * batch_size, :]
                split_y = y_clss[i * batch_size : (i + 1) * batch_size]
                splits.append(np.hstack((split_X, split_y.reshape(-1, 1))))
            class_partitions.append(splits)

        # -------------------
        # consolidate splits into ndarrays
        # -------------------

        grouped = []
        for split in range(n_splits):
            parts = []
            for part in class_partitions:
                parts.append(part[split])
            grouped.append(parts)

        splits = []
        for group in grouped:
            splits.append(np.vstack(group))

        return splits

    @staticmethod
    def _make_model_list(learner, n) -> List:
        """
        Intances n shadow models, copies of the input parameter learner
        """
        try:
            if isinstance(learner, tf.keras.models.Model):
                models = [copy(learner) for _ in range(n)]
        except NameError:
            print("using sklearn shadow models")
            pass

        if isinstance(learner, BaseEstimator):
            models = [clone(learner) for _ in range(n)]

        return models

    def train_predict_shadows(self, **fit_kwargs):
        """
        "in" : 1
        "out" : 0
        """

        # TRAIN and predict
        results = []
        for model, data_subset in tqdm_notebook(zip(self.models, self._splits)):
            X = data_subset[:, :-1]
            y = data_subset[:, -1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

            model.fit(X_train, y_train, **fit_kwargs)
            # data IN training set labelet 1
            y_train = y_train.reshape(-1, 1)
            predict_in = model.predict_proba(X_train)
            res_in = np.hstack((predict_in, y_train, np.ones_like(y_train)))

            # data OUT of training set, labeled 0
            y_test = y_test.reshape(-1, 1)
            predict_out = model.predict_proba(X_test)
            print(predict_out)
            res_out = np.hstack((predict_out, y_test, np.zeros_like(y_test)))

            # concat in single array
            model_results = np.vstack((res_in, res_out))
            results.append(model_results)

        results = np.vstack(results)
        return results

    def __repr__(self):
        rep = (
            f"Shadow models: {self.n_models}, {self.learner.__class__}\n"
            f"lengths of data splits : {[len(s) for s in self._splits]}"
        )
        return rep

In [5]:
logisticRegr = LogisticRegression()
sh = ShadowModels(x, y, 5, 3, logisticRegr)
shadow_data = sh.results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[[0.41679353 0.58320647]
 [0.8788776  0.1211224 ]
 [0.72409235 0.27590765]
 ...
 [0.82369893 0.17630107]
 [0.76877759 0.23122241]
 [0.8397024  0.1602976 ]]
[[0.0102458  0.9897542 ]
 [0.85825043 0.14174957]
 [0.82621174 0.17378826]
 ...
 [0.85194719 0.14805281]
 [0.93082198 0.06917802]
 [0.77212431 0.22787569]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[[0.47360144 0.52639856]
 [0.1618165  0.8381835 ]
 [0.96057355 0.03942645]
 ...
 [0.83368732 0.16631268]
 [0.25291645 0.74708355]
 [0.8495095  0.1504905 ]]
[[0.85997323 0.14002677]
 [0.91744456 0.08255544]
 [0.71736058 0.28263942]
 ...
 [0.70846999 0.29153001]
 [0.77586532 0.22413468]
 [0.91992176 0.08007824]]
[[0.92274869 0.07725131]
 [0.84344514 0.15655486]
 [0.84046922 0.15953078]
 ...
 [0.09023078 0.90976922]
 [0.86272243 0.13727757]
 [0.80712885 0.19287115]]


In [6]:
shadow_data

array([[0.69040221, 0.30959779, 0.        , 1.        ],
       [0.76449985, 0.23550015, 0.        , 1.        ],
       [0.79814186, 0.20185814, 0.        , 1.        ],
       ...,
       [0.09023078, 0.90976922, 1.        , 0.        ],
       [0.86272243, 0.13727757, 0.        , 0.        ],
       [0.80712885, 0.19287115, 0.        , 0.        ]])

In [7]:
class AttackModels:
    def __init__(self, target_classes, attack_learner):
        """
        Attacker models to learn class membership from shadow data.
        Parameters
        ----------
        target_classes: int
            number of classes that the target model can predict
        attack_learning: learner
            trainable learner to model memebership from shadow data.
            The learner its cloned into n models, one for each target class,
            and each model is trained on a class subset of the shadow data.
        Returns
        -------
        AttackModels class instance
        """
        self.target_classes = target_classes
        self.attack_learner = attack_learner
        # 1 model for each class
        self.attack_models = [clone(self.attack_learner) for _ in range(target_classes)]

        self._fited = False

    @staticmethod
    def _update_learner_params(learner, **learner_params) -> None:
        # safety check if dict is well formed
        for k in learner_params.keys():
            if not hasattr(learner, k):
                raise AttributeError(
                    f"Learner parameter {k} is not an attribute of {learner.__class__}"
                )

        # update learner params
        learner.__dict__.update(**learner_params)

    def fit(self, shadow_data, **learner_kwargs) -> None:
        """
        Trains `attack_models` with `shadow_data`. Each model is trained with
        with a subset of the same class of `shadow_data`.
        Parameters
        ----------
        shadow_data: np.ndarray
            Shadow data. Results from `ShadowModels`.
            Last column (`[:,-1]`) must be the membership label of the shadow
            prediction, where 1 means that the record was present in the 
            shadow training set ('in') and 0 if the recored was in the test
            set ('out').
            Second last column (`[:,-2]`) must be the data class. this will
            be used as grouper to split the data for each attack model.
            The rest of the columns are the class probability vector
            predicted by the shadow model.
        Returns
        -------
        None
        TODO
        ----
            Tweak model params with something like **learner_kwargs
            cross-validate
            grid search?
        """
        # split data into subsets, n == target_classes
        membership_label = shadow_data[:, -1]
        class_label = shadow_data[:, -2]
        data = shadow_data[:, :-2]
        for i, model in enumerate(self.attack_models):
            X = data[class_label == i]
            y = membership_label[class_label == i]

            # update model params
            self._update_learner_params(model, **learner_kwargs)
            # train model
            model.fit(X, y)

        self._fited = True

    def predict(self, X, y, batch=False) -> np.ndarray:
        """
        Predicts if `X` is real member of `y` in the attacked
        private training set.
        Parameters
        ----------
        X: np.ndarray
            Probability vector result from target model
        y: int, np.ndarray
            estimated class of the data record used to get `X`
        """
        if not self._fited:
            print("Must run `fit` method first")
            return

        if not batch:
            model_cls = y
            model = self.attack_models[model_cls]
            prob_vec = model.predict_proba(X)

            if y == np.argmax(prob_vec) and np.argmax(prob_vec) == 1:
                return 1

            else:
                return 0

        elif batch:

            model_classes = np.unique(y)
            res = []
            for model_cls in model_classes:
                X_cls = X[y == model_cls]
                model = self.attack_models[model_cls]
                attack_res = model.predict_proba(X_cls)
                res.append(attack_res)

            return np.concatenate(res)

In [8]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=2, attack_learner=model)
attacker.fit(shadow_data)

# **Membership attack on target Decision Tree**

In [62]:
from sklearn.svm import SVC
df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/adult census/adult.data',header= None,delimiter=",")
df[14] = np.where(df[14]==' <=50K', 0, 1)
x = df.drop(14,axis = 1 )
y = df[14]
x = pd.get_dummies(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.34, random_state=42)
clf = DecisionTreeClassifier(random_state=0,max_leaf_nodes = 2000)
#clf = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, probability = True))
clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


DecisionTreeClassifier(max_leaf_nodes=2000, random_state=0)

In [63]:
y_pred_train = clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)


accuracy on train data is 97.28245695672406
accuracy on test data is 83.09999096739229


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [64]:
X_in = clf.predict_proba(x_train)
res_in = attacker.predict(X_in, y_train, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [65]:
X_in

array([[0.98709677, 0.01290323],
       [0.99557352, 0.00442648],
       [0.99557352, 0.00442648],
       ...,
       [0.99557352, 0.00442648],
       [0.90588235, 0.09411765],
       [1.        , 0.        ]])

In [66]:
res_in

array([[0.03, 0.97],
       [0.89, 0.11],
       [0.89, 0.11],
       ...,
       [0.1 , 0.9 ],
       [0.91, 0.09],
       [0.1 , 0.9 ]])

In [67]:
X_out = clf.predict_proba(x_test)
res_out = attacker.predict(X_out, y_test, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [68]:
X_out

array([[0.99557352, 0.00442648],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.97532315, 0.02467685],
       [0.99557352, 0.00442648],
       [1.        , 0.        ]])

In [69]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = np.concatenate((np.argmax(res_in, axis=1), np.argmax(res_out, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [70]:
precision_score(y_true, y_pred)

0.6654386683039978

In [71]:
recall_score(y_true, y_pred)

0.22694276407631456

In [72]:
f1_score(y_true, y_pred)

0.33845726777473195

# **Membership attack on target Random Forest**

In [20]:
from sklearn.ensemble import RandomForestClassifier
r_clf = RandomForestClassifier(max_depth=10, n_estimators = 2, random_state=0)
r_clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


RandomForestClassifier(max_depth=10, n_estimators=2, random_state=0)

In [21]:
y_pred_train = r_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = r_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 85.82131223825034
accuracy on test data is 84.52714298618011


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [22]:
X_in = r_clf.predict_proba(x_train)
res_input = attacker.predict(X_in, y_train, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [23]:
X_in

array([[0.83971705, 0.16028295],
       [0.98548538, 0.01451462],
       [0.98548538, 0.01451462],
       ...,
       [0.98548538, 0.01451462],
       [0.83971705, 0.16028295],
       [0.84149272, 0.15850728]])

In [24]:
res_input

array([[0.15, 0.85],
       [0.02, 0.98],
       [0.02, 0.98],
       ...,
       [0.61, 0.39],
       [0.7 , 0.3 ],
       [0.61, 0.39]])

In [25]:
X_out = r_clf.predict_proba(x_test)
res_output = attacker.predict(X_out, y_test, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [26]:
res_output

array([[0.78, 0.22],
       [0.94, 0.06],
       [0.7 , 0.3 ],
       ...,
       [0.74, 0.26],
       [0.27, 0.73],
       [0.74, 0.26]])

In [27]:
X_out

array([[0.98029984, 0.01970016],
       [0.54468698, 0.45531302],
       [0.28070029, 0.71929971],
       ...,
       [0.99025628, 0.00974372],
       [0.99057818, 0.00942182],
       [0.96483051, 0.03516949]])

In [28]:
y_pred = np.concatenate((np.argmax(res_input, axis=1), np.argmax(res_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [29]:
precision_score(y_true, y_pred)

0.6585779327714811

In [30]:
recall_score(y_true, y_pred)

0.632712889716147

In [31]:
f1_score(y_true, y_pred)

0.6453863679513955

# Membership inference attack on stacking classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr', make_pipeline(StandardScaler(),
                          LinearSVC(random_state=42)))
]
s_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

s_clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only sup

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(random_state=42))]))],
                   final_estimator=LogisticRegression())

In [33]:
y_pred_train = s_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = s_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


accuracy on train data is 91.35411819450907
accuracy on test data is 85.76461024297714


In [34]:
X_in = s_clf.predict_proba(x_train)
r_input = attacker.predict(X_in, y_train, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [35]:
X_out = s_clf.predict_proba(x_test)
r_output = attacker.predict(X_out, y_test, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [36]:
y_pred = np.concatenate((np.argmax(r_input, axis=1), np.argmax(r_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [37]:
precision_score(y_true, y_pred)

0.6614880367092757

In [38]:
recall_score(y_true, y_pred)

0.46956724057701255

In [39]:
f1_score(y_true, y_pred)

0.5492447952102326